<a href="https://colab.research.google.com/github/AlexandreMenezesSa/pyspark/blob/main/spark_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark sql**

In [1]:
# instalar pyspark
!pip install pyspark

In [2]:
#faz importação dos pacotes e inicia a sessão
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
from pyspark.sql.types import *
from pyspark.sql import functions as Func
from pyspark.sql.functions import *
spark = SparkSession.builder.master("local[*]").appName("PySpark").getOrCreate()

In [35]:
#mostrar bancos de dados e tabelas
spark.sql("show databases").show()


+---------+
|namespace|
+---------+
|  default|
|     desp|
+---------+



In [18]:
#drop banco de dados caso exista
spark.sql("DROP DATABASE IF EXISTS desp CASCADE")


DataFrame[]

In [19]:
#criar banco de dados
spark.sql("create database desp")
spark.sql("use desp").show()

++
||
++
++



In [21]:
#criar tabela gerenciada
arqschema = "id INT, nome STRING, status STRING, cidade STRING, vendas INT, data STRING"
despachantes = spark.read.csv("despachantes.csv", header=False, schema=arqschema)
despachantes.write.saveAsTable("Despachantes")

In [22]:
#mostrar que a tabela existe
spark.sql("select * from despachantes").show()

+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



In [34]:
#mostra tabela
spark.sql("show tables").show()

+---------+---------------+-----------+
|namespace|      tableName|isTemporary|
+---------+---------------+-----------+
|  default|despachantes_ng|      false|
+---------+---------------+-----------+



In [24]:

#mudar banco de dados
spark.sql("use default").show()


++
||
++
++



In [41]:
#voltar banco de dados
spark.sql("use desp").show()

++
||
++
++



In [42]:
#criar tabela não gerenciada
#salvamos novamente no formato parquet, em outro diretorio
despachantes.write.format("parquet").mode("overwrite").save("desparquet")
#informar o caminho
despachantes.write.option("path","desparquet").saveAsTable("despachantes_ng")

AnalysisException: [TABLE_OR_VIEW_ALREADY_EXISTS] Cannot create table or view `spark_catalog`.`desp`.`despachantes_ng` because it already exists.
Choose a different name, drop or replace the existing object, or add the IF NOT EXISTS clause to tolerate pre-existing objects.

In [43]:
#como saber se uma tabela é gerenciada ou não?
#podemos observar que despachantes não mostra o caminho
spark.sql("show create table despachantes").show(truncate=False)


+-------------------------------------------------------------------------------------------------------------------------------------------------------------+
|createtab_stmt                                                                                                                                               |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------+
|CREATE TABLE spark_catalog.desp.despachantes (\n  id INT,\n  nome STRING,\n  status STRING,\n  cidade STRING,\n  vendas INT,\n  data STRING)\nUSING parquet\n|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------+



In [44]:
#despachantes ng mostra, indicando que é não gerenciada
spark.sql("show create table Despachantes_ng").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|createtab_stmt                                                                                                                                                                                                               |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|CREATE TABLE spark_catalog.desp.Despachantes_ng (\n  id INT,\n  nome STRING,\n  status STRING,\n  cidade STRING,\n  vendas INT,\n  data STRING)\nUSING parquet\nLOCATION 'file:/content/spark-warehouse/desp.db/desparquet'\n|
+-------------------------------------------------------------------------------------------------------

In [45]:
#outra forma:
spark.catalog.listTables()

[Table(name='despachantes', catalog='spark_catalog', namespace=['desp'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='despachantes_ng', catalog='spark_catalog', namespace=['desp'], description=None, tableType='EXTERNAL', isTemporary=False)]